In [1]:
import Pkg; Pkg.add("ITensors")
import Pkg; Pkg.add("ITensorMPS")
import Pkg; Pkg.add("BlockArrays")
import Pkg; Pkg.add("LinearAlgebra")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
let
    c=1
  N = 14
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 5 # number of sweeps is 5
  maxdim = [10] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi = dmrg(H,psi0;nsweeps,maxdim,cutoff)
psidag=prime(dag(psi),linkinds(psi))
orthogonalize!(psi,2)
rho=prime(psi[1],sites[1])*dag(psi[1])
    
orthogonalize!(psi,10)
rhoa=prime(psi[10],sites[10])*dag(psi[10])
    
rho1=rho*rhoa
#for i=3:4
 #rho1 *= psi[i]
 #rho1 *= psidag[i]
#end

#@show rho1
@show inds(rho1) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 

rho1_PT = swapinds(rho1,inds(rho1)[1],inds(rho1)[3])
    
a1=0
N1=0
d1,u1 = eigen(rho1_PT)

@show diag(real(d1));        
for j=1:length(diag(real(d1)))
   a1+=diag(real(d1))[j]
end
@show a1;
    
    
for j=1:length(diag(real(d1)))
   N1+=0.5*(abs(diag(real(d1))[j])-diag(real(d1))[j])       
    end   
L1= log(1+2*N1)

    @show N1;
   @show L1;

    return 
end    

After sweep 1 energy=-6.024263071950934  maxlinkdim=10 maxerr=8.69E-04 time=19.831
After sweep 2 energy=-6.026681819257319  maxlinkdim=10 maxerr=4.12E-06 time=0.021
After sweep 3 energy=-6.026682002159758  maxlinkdim=10 maxerr=3.43E-06 time=0.071
After sweep 4 energy=-6.026682007225125  maxlinkdim=10 maxerr=3.42E-06 time=0.062
After sweep 5 energy=-6.026682007427498  maxlinkdim=10 maxerr=3.42E-06 time=0.051
inds(rho1) = ((dim=2|id=783|"S=1/2,Site,n=1")', (dim=2|id=783|"S=1/2,Site,n=1"), (dim=2|id=165|"S=1/2,Site,n=10")', (dim=2|id=165|"S=1/2,Site,n=10"))
diag(real(d1)) = [-0.4999999997676276, 0.4999847562034268, 0.49999999976762705, 0.500015243796574]
a1 = 1.0000000000000002
N1 = 0.4999999997676276
L1 = 0.6931471803275729


In [3]:
using ITensors, ITensorMPS

let
    N = 14
    sites = siteinds("S=1/2", N)

    # Construct the Hamiltonian
    os = OpSum()
    for j in 1:N-1
        os += 0.5, "S+", j, "S-", j + 1
        os += 0.5, "S-", j, "S+", j + 1
        os += "Sz", j, "Sz", j + 1
    end
    H = MPO(os, sites)

    # DMRG parameters
    nsweeps = 5
    maxdim = [10]
    cutoff = [1E-10]

    # Initialize random MPS
    psi0 = random_mps(sites; linkdims=10)

    # Perform DMRG
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)

    # Specify sites A, B, C, D
    sites_to_keep = [1, 2, 5, 6]

    # Compute reduced density matrix for the specified sites
    rho = reduced_density_matrix(psi, sites_to_keep)

    # Partial transposes over each site
    rho_PT_A = swapinds(rho, inds(rho)[1], inds(rho)[3]) # Partial transpose over site A
    rho_PT_B = swapinds(rho, inds(rho)[1], inds(rho)[2]) # Partial transpose over site B
    rho_PT_C = swapinds(rho, inds(rho)[3], inds(rho)[4]) # Partial transpose over site C
    rho_PT_D = swapinds(rho, inds(rho)[2], inds(rho)[4]) # Partial transpose over site D

    # Function to compute log negativity
    function log_negativity(rho_PT)
        d1, u1 = eigen(rho_PT)
        # Calculate the absolute values of negative eigenvalues
        negativity = sum(max(0, -eigval) for eigval in diag(real(d1)))
        return log(1 + 2 * negativity)
    end

    # Calculate log negativities for each partial transpose
    L_A = log_negativity(rho_PT_A)
    L_B = log_negativity(rho_PT_B)
    L_C = log_negativity(rho_PT_C)
    L_D = log_negativity(rho_PT_D)

    # Seventh root of the product of log negativities
    L_product = L_A + L_B + L_C + L_D # Note: sum of log negativities
    seventh_root = exp(L_product / 7) # Calculate the seventh root

    # Display results
    @show L_A
    @show L_B
    @show L_C
    @show L_D
    @show seventh_root

    return 
end


After sweep 1 energy=-6.025747914044384  maxlinkdim=10 maxerr=1.55E-03 time=0.019
After sweep 2 energy=-6.026681949173851  maxlinkdim=10 maxerr=3.72E-06 time=0.020
After sweep 3 energy=-6.026682006889255  maxlinkdim=10 maxerr=3.43E-06 time=0.068
After sweep 4 energy=-6.026682007421956  maxlinkdim=10 maxerr=3.42E-06 time=0.043
After sweep 5 energy=-6.026682007435503  maxlinkdim=10 maxerr=3.42E-06 time=0.037


LoadError: UndefVarError: `reduced_density_matrix` not defined

In [4]:
let
    c=1
  N = 14
  sites = siteinds("S=1/2",N)

  os = OpSum()
  for j=1:N-1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os += "Sz",j,"Sz",j+1
  end
  H = MPO(os,sites)

  nsweeps = 5 # number of sweeps is 5
  maxdim = [10, 100, 1000, 10000, 25580] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error

  psi0 = random_mps(sites;linkdims=10)

  energy,psi = dmrg(H,psi0;nsweeps,maxdim,cutoff)
        
orthogonalize!(psi,2)

# Reduced density matrix between sites 1, 2, 3, 4 in the order 1234
    
psidag=prime(dag(psi),linkinds(psi))

rho1=prime(psi[1],sites[1])*dag(psi[1])
for i=2:4
    rho1 *= psi[i]
    rho1 *= psidag[i]
end

#rho1 *= prime(psi[4],linkinds(psi,5))*prime(psidag[4],sites[4])

#for i=3:4
 #rho1 *= psi[i]
 #rho1 *= psidag[i]
#end

#@show rho1
#@show inds(rho1) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 

rho1_PT = swapinds(rho1,inds(rho1)[1],inds(rho1)[3])
    
    

    

############################################################################################
    
# Reduced density matrix between sites 1, 2, 3, 4 in the order 4123
orthogonalize!(psi,c+3)
ket2=psi[c+3]
ket2 *= psi[c]*psi[c+1]*psi[c+2]   
bra2 = dag(ket2)
rho2 = prime(ket2,"Site")*bra2 
#@show inds(rho2) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho2_PT = swapinds(rho2,inds(rho2)[1],inds(rho2)[3])

    
    
    
    
    
    
############################################################################################

# Reduced density matrix between sites 1, 2, 3, 4 in the order 3124
orthogonalize!(psi,c+4)
ket3=psi[c+4]
ket3 = psi[c]*psi[c+1]*psi[c+3]   
bra3 = dag(ket3)
rho3 = prime(ket3,"Site")*bra3 
#@show inds(rho3) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho3_PT = swapinds(rho3,inds(rho3)[1],inds(rho3)[3])  
    
    
    
    
    
    
    
    
############################################################################################
  
orthogonalize!(psi,c)
orthogonalize!(psi,c+1)
orthogonalize!(psi,c+2)
ket4 = psi[c]*psi[c+1]*psi[c+2]   
bra4 = dag(ket4)
rho4 = prime(ket4,"Site")*bra4 
#@show inds(rho4) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho4_PT = swapinds(rho4,inds(rho4)[1],inds(rho4)[3])     
    
    
    
    
    
    
    
    
############################################################################################
   
orthogonalize!(psi,c)
ket5 = psi[c]*psi[c+1]   
bra5 = dag(ket5)
rho5 = prime(ket5,"Site")*bra5 
@show inds(rho5) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho5_PT = swapinds(rho5,inds(rho5)[1],inds(rho5)[3])    
  
    
    
    
    
    
    
    
############################################################################################
   
orthogonalize!(psi,c+3)
ket6 = psi[c]*psi[c+2]   
bra6 = dag(ket6)
rho6 = prime(ket6,"Site")*bra6 
#@show inds(rho6) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho6_PT = swapinds(rho6,inds(rho6)[1],inds(rho6)[3])         
    
    
    
    
    
    
    
    
############################################################################################

orthogonalize!(psi,c+1)
orthogonalize!(psi,c+2)
ket7 = psi[c]*psi[c+3]   
bra7 = dag(ket7)
rho7 = prime(ket7,"Site")*bra7 
@show inds(rho7) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho7_PT = swapinds(rho7,inds(rho7)[1],inds(rho7)[3])
  
    
    
    
    
    

############################################################################################
   
a1=0
a2=0
a3=0
a4=0
a5=0
a6=0
a7=0
N1=0
N2=0
N3=0
N4=0
N5=0
N6=0
N7=0
d1,u1 = eigen(rho1_PT)
d2,u2 = eigen(rho2_PT)
d3,u3 = eigen(rho3_PT)
d4,u4 = eigen(rho4_PT)
d5,u5 = eigen(rho5_PT)
d6,u6 = eigen(rho6_PT)
d7,u7 = eigen(rho7_PT)
#@show diag(real(d1));        
#@show diag(real(d2));
#@show diag(real(d3));        
#@show diag(real(d4));
#@show diag(real(d5));        
#@show diag(real(d6));
@show diag(real(d7));# if this reduced density matrix fails the PPT criterion, it will show some negative eigenvalues signaling mixed state entanglement.
for j=1:length(diag(real(d1)))
   a1+=diag(real(d1))[j]
end
for j=1:length(diag(real(d2)))
   a2+=diag(real(d2))[j]
end
for j=1:length(diag(real(d3)))
   a3+=diag(real(d3))[j]
end
for j=1:length(diag(real(d4)))
   a4+=diag(real(d4))[j]
end
for j=1:length(diag(real(d5)))
   a5+=diag(real(d5))[j]
end
for j=1:length(diag(real(d6)))
   a6+=diag(real(d6))[j]
end
for j=1:length(diag(real(d7)))
   a7+=diag(real(d7))[j]
end
    @show a1;
    @show a2;
    @show a3;
    @show a4;
    @show a5;
    @show a6;
    @show a7;
    
for j=1:length(diag(real(d1)))
   N1+=0.5*(abs(diag(real(d1))[j])-diag(real(d1))[j])       
    end   
for j=1:length(diag(real(d2)))
   N2+=0.5*(abs(diag(real(d2))[j])-diag(real(d2))[j])
    end
for j=1:length(diag(real(d3)))
   N3+=0.5*(abs(diag(real(d3))[j])-diag(real(d3))[j])
    end
for j=1:length(diag(real(d4)))
   N4+=0.5*(abs(diag(real(d4))[j])-diag(real(d4))[j])
    end
for j=1:length(diag(real(d5)))
   N5+=0.5*(abs(diag(real(d5))[j])-diag(real(d5))[j])
    end
for j=1:length(diag(real(d6)))
   N6+=0.5*(abs(diag(real(d6))[j])-diag(real(d6))[j])
    end
for j=1:length(diag(real(d7)))
   N6+=0.5*(abs(diag(real(d7))[j])-diag(real(d7))[j])
    end
L1= log(1+2*N1)
L2= log(1+2*N2)
L3= log(1+2*N3)
L4= log(1+2*N4)
L5= log(1+2*N5)
L6= log(1+2*N6)
L7= log(1+2*N7)
    @show N1;
    @show L1;
    @show N2;
    @show L2;
    @show N3;
    @show L3;
    @show N4;
    @show L4;
    @show N5;
    @show L5;
    @show N6;
    @show L6;
    @show N7;
    @show L7;
    return 
end

After sweep 1 energy=-6.0252512293662335  maxlinkdim=10 maxerr=1.40E-03 time=0.031
After sweep 2 energy=-6.026724625582505  maxlinkdim=32 maxerr=9.02E-11 time=0.058
After sweep 3 energy=-6.026724660088863  maxlinkdim=31 maxerr=9.80E-11 time=0.081
After sweep 4 energy=-6.026724660088037  maxlinkdim=31 maxerr=9.03E-11 time=0.081
After sweep 5 energy=-6.0267246600878455  maxlinkdim=31 maxerr=9.04E-11 time=0.070
inds(rho5) = ((dim=2|id=656|"S=1/2,Site,n=1")', (dim=2|id=0|"S=1/2,Site,n=2")', (dim=2|id=656|"S=1/2,Site,n=1"), (dim=2|id=0|"S=1/2,Site,n=2"))
inds(rho7) = ((dim=2|id=656|"S=1/2,Site,n=1")', (dim=2|id=211|"S=1/2,Site,n=4")', (dim=2|id=656|"S=1/2,Site,n=1"), (dim=2|id=211|"S=1/2,Site,n=4"))
diag(real(d7)) = [3.9999993631087793, 3.9999993631087793, 4.000000636891217, 4.000000636891218]
a1 = 0.9999999999999999
a2 = 0.832375681772019
a3 = 32.0
a4 = 0.8972757345854661
a5 = 1.0
a6 = 15.999999999999995
a7 = 15.999999999999993
N1 = 1.3122950520351069e-16
L1 = 2.2204460492503128e-16
N2 = 3

In [5]:
function partial_transpose_per(x, sys, dims)
    n = length(dims)
    d = prod(dims)
    s = n - sys + 1
    p = collect(1:2n)
    p[s] = n + s
    p[n + s] = s
    rdims = reverse(dims)
    r = reshape(x, (rdims..., rdims...))
    return reshape(permutedims(r,p),(d,d))
end


partial_transpose_per (generic function with 1 method)

In [6]:
#function partial_transpose(A::MPO, sites)
 # A = copy(A)
 # for n in sites
 #   A[n] = swapinds(A[n], siteinds(A, n)...)
 # end
 # return A
#end

In [7]:
#function ptrace(σ::Operator, out)
 #   res = ptrace(Array(σ),out,dims(σ))
  #  return Operator(res[1],res[2])
#end

In [8]:
using ITensors, ITensorMPS

let
    c=1
  N = 14

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=1",N)

  # Couplings between spin-half and
  # spin-one sites:
  Jho = 1.0 # half-one coupling
  Jhh = 0.5 # half-half coupling
  Joo = 0.5 # one-one coupling

  os = OpSum()
  for j=1:N-1
    os += 0.5*Jho,"S+",j,"S-",j+1
    os += 0.5*Jho,"S-",j,"S+",j+1
    os += Jho,"Sz",j,"Sz",j+1
  end
  for j=1:2:N-2
    os += 0.5*Jhh,"S+",j,"S-",j+2
    os += 0.5*Jhh,"S-",j,"S+",j+2
    os += Jhh,"Sz",j,"Sz",j+2
  end
  for j=2:2:N-2
    os += 0.5*Joo,"S+",j,"S-",j+2
    os += 0.5*Joo,"S-",j,"S+",j+2
    os += Joo,"Sz",j,"Sz",j+2
  end
    os += 0.5*Jho,"S+",N,"S-",1
    os += 0.5*Jho,"S-",N,"S+",1
    os += Jho,"Sz",N,"Sz",1
    
  H = MPO(os,sites)

nsweeps = 10
maxdim = [10,20,40,80,100,140,180,200]
cutoff = [1E-8]

  psi0 = random_mps(sites;linkdims=2)

  energy,psi = dmrg(H,psi0;nsweeps,maxdim,cutoff)

orthogonalize!(psi,c)
ket1 = psi[c]*psi[c+1]   
bra1 = dag(ket1)
rho1 = prime(ket1,"Site")*bra1     # if the two sites are not adjacent, something else needs to be added to the construction of the red. density matrix.

rho1_PT = swapinds(rho1,inds(rho1)[1],inds(rho1)[3])
    
    
orthogonalize!(psi,c+6)
ket2 = psi[c+6]*psi[c+7]   
bra2 = dag(ket2)
rho2 = prime(ket2,"Site")*bra2 
@show inds(rho2) # this shows that inds(rho1)[2] and inds(rho1)[4] belong to the second site c+7. Suppose you now want to partial-transpose with respect to the basis at this site, then do 
rho2_PT = swapinds(rho2,inds(rho2)[1],inds(rho2)[3])
    
a1=0
a2=0
N1=0
N2=0
d1,u1 = eigen(rho1_PT)
d2,u2 = eigen(rho2_PT)
@show diag(real(d1));        
@show diag(real(d2));  # if this reduced density matrix fails the PPT criterion, it will show some negative eigenvalues signaling mixed state entanglement.
for j=1:length(diag(real(d1)))
   a1+=diag(real(d1))[j]
   a2+=diag(real(d2))[j]
end
    @show a1;
    @show a2;
    
for j=1:length(diag(real(d1)))
   N1+=0.5*(abs(diag(real(d1))[j])-diag(real(d1))[j])
   N2+=0.5*(abs(diag(real(d2))[j])-diag(real(d2))[j])
    end
L1= log(1+2*N1)
L2= log(1+2*N2)
    @show N1;
    @show L1;
    @show N2;
    @show L2;
    return 
end

After sweep 1 energy=-8.596228240283427  maxlinkdim=10 maxerr=2.23E-04 time=0.019
After sweep 2 energy=-8.97699784754007  maxlinkdim=20 maxerr=2.95E-04 time=0.116
After sweep 3 energy=-9.082103296721861  maxlinkdim=40 maxerr=2.02E-04 time=0.272
After sweep 4 energy=-9.09363183916874  maxlinkdim=80 maxerr=3.26E-05 time=1.050
After sweep 5 energy=-9.094366634067969  maxlinkdim=100 maxerr=1.20E-05 time=0.782
After sweep 6 energy=-9.094527822753696  maxlinkdim=140 maxerr=1.82E-06 time=1.128
After sweep 7 energy=-9.094550116658299  maxlinkdim=180 maxerr=2.48E-07 time=1.561
After sweep 8 energy=-9.094553069352262  maxlinkdim=200 maxerr=9.10E-08 time=1.807
After sweep 9 energy=-9.094553374274799  maxlinkdim=200 maxerr=8.79E-08 time=2.260
After sweep 10 energy=-9.094553419704722  maxlinkdim=200 maxerr=8.52E-08 time=1.746
inds(rho2) = ((dim=2|id=301|"S=1/2,Site,n=7")', (dim=3|id=214|"S=1,Site,n=8")', (dim=2|id=301|"S=1/2,Site,n=7"), (dim=3|id=214|"S=1,Site,n=8"))
diag(real(d1)) = [-0.1159545991

In [9]:
#@show inds(T)

In [10]:
i = Index(3,"index_i")
j = Index(2,"index_j")
k = Index(4,"index_k")

T = ITensor(i,j,k)

ITensor ord=3 (dim=3|id=443|"index_i") (dim=2|id=814|"index_j") (dim=4|id=518|"index_k")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [11]:
?random_mps

search: random_mps random_mpo random_itensor



```
random_mps(eltype::Type{<:Number}, sites::Vector{<:Index}; linkdims=1)
```

Construct a random MPS with link dimension `linkdims` of type `eltype`.

`linkdims` can also accept a `Vector{Int}` with `length(linkdims) == length(sites) - 1` for constructing an MPS with non-uniform bond dimension.

---

```
random_mps(sites::Vector{<:Index}; linkdims=1)
random_mps(eltype::Type{<:Number}, sites::Vector{<:Index}; linkdims=1)
```

Construct a random MPS with link dimension `linkdims` which by default has element type `Float64`.

`linkdims` can also accept a `Vector{Int}` with `length(linkdims) == length(sites) - 1` for constructing an MPS with non-uniform bond dimension.

---

```
random_mps(sites::Vector{<:Index}, state; linkdims=1)
```

Construct a real, random MPS with link dimension `linkdims`, made by randomizing an initial product state specified by `state`. This version of `random_mps` is necessary when creating QN-conserving random MPS (consisting of QNITensors). The initial `state` array provided determines the total QN of the resulting random MPS.


In [12]:
sites = siteinds("S=1/2",10)

10-element Vector{Index{Int64}}:
 (dim=2|id=662|"S=1/2,Site,n=1")
 (dim=2|id=803|"S=1/2,Site,n=2")
 (dim=2|id=349|"S=1/2,Site,n=3")
 (dim=2|id=76|"S=1/2,Site,n=4")
 (dim=2|id=502|"S=1/2,Site,n=5")
 (dim=2|id=636|"S=1/2,Site,n=6")
 (dim=2|id=550|"S=1/2,Site,n=7")
 (dim=2|id=861|"S=1/2,Site,n=8")
 (dim=2|id=237|"S=1/2,Site,n=9")
 (dim=2|id=439|"S=1/2,Site,n=10")

In [13]:
using ITensors, ITensorMPS

let
    c=1
  N = 14

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=1",N)

  # Couplings between spin-half and
  # spin-one sites:
  Jho = 1.0 # half-one coupling
  Jhh = 0.5 # half-half coupling
  Joo = 0.5 # one-one coupling

  os = OpSum()
  for j=1:N-1
    os += 0.5*Jho,"S+",j,"S-",j+1
    os += 0.5*Jho,"S-",j,"S+",j+1
    os += Jho,"Sz",j,"Sz",j+1
  end
    os += 0.5*Jho,"S+",14,"S-",1
    os += 0.5*Jho,"S-",14,"S+",1
    os += Jho,"Sz",14,"Sz",1
  for j=1:2:N-2
    os += 0.5*Jhh,"S+",j,"S-",j+2
    os += 0.5*Jhh,"S-",j,"S+",j+2
    os += Jhh,"Sz",j,"Sz",j+2
  end
  for j=2:2:N-2
    os += 0.5*Joo,"S+",j,"S-",j+2
    os += 0.5*Joo,"S-",j,"S+",j+2
    os += Joo,"Sz",j,"Sz",j+2
  end
    os += 0.5*Jho,"S+",N,"S-",1
    os += 0.5*Jho,"S-",N,"S+",1
    os += Jho,"Sz",N,"Sz",1
    
    
  H = MPO(os,sites)

  nsweeps = 10
  maxdim = [10,20,40,80,100,140,180,200]
    cutoff = [1E-8]

  psi0 = random_mps(sites;linkdims=4)
    
  energy,psi = dmrg(H,psi0;nsweeps,maxdim,cutoff)

   orthogonalize!(psi,c)
    ket = psi[c]*psi[c+1]
    bra = dag(ket)
    rho = prime(ket,"Site")*bra
    
    return @show rho;
end
    

After sweep 1 energy=-8.946267387797501  maxlinkdim=10 maxerr=4.10E-04 time=0.027
After sweep 2 energy=-9.806259711364138  maxlinkdim=20 maxerr=2.91E-04 time=0.133
After sweep 3 energy=-9.900787590080899  maxlinkdim=40 maxerr=2.02E-04 time=0.216
After sweep 4 energy=-9.927368771597356  maxlinkdim=80 maxerr=5.15E-05 time=0.543
After sweep 5 energy=-9.931366426380054  maxlinkdim=100 maxerr=2.84E-05 time=0.837
After sweep 6 energy=-9.932363278454858  maxlinkdim=140 maxerr=4.35E-06 time=1.263
After sweep 7 energy=-9.932573460913297  maxlinkdim=180 maxerr=5.20E-07 time=1.727
After sweep 8 energy=-9.932616880184721  maxlinkdim=200 maxerr=1.88E-07 time=2.320
After sweep 9 energy=-9.932626129093926  maxlinkdim=200 maxerr=1.42E-07 time=1.803
After sweep 10 energy=-9.932627951384816  maxlinkdim=200 maxerr=1.31E-07 time=1.930
rho = ITensor ord=4
Dim 1: (dim=2|id=646|"S=1/2,Site,n=1")'
Dim 2: (dim=3|id=112|"S=1,Site,n=2")'
Dim 3: (dim=2|id=646|"S=1/2,Site,n=1")
Dim 4: (dim=3|id=112|"S=1,Site,n=2")

ITensor ord=4 (dim=2|id=646|"S=1/2,Site,n=1")' (dim=3|id=112|"S=1,Site,n=2")' (dim=2|id=646|"S=1/2,Site,n=1") (dim=3|id=112|"S=1,Site,n=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [14]:
r = rand(36,36,36)

36×36×36 Array{Float64, 3}:
[:, :, 1] =
 0.673292   0.151132   0.387882    …  0.246315   0.516045   0.354407
 0.256919   0.557478   0.670395       0.629406   0.407153   0.657824
 0.534967   0.141586   0.478585       0.934135   0.189038   0.610413
 0.284075   0.341061   0.00151322     0.0730159  0.916061   0.723393
 0.229107   0.412506   0.24909        0.0493924  0.755759   0.569338
 0.785899   0.372507   0.837035    …  0.153244   0.62781    0.423974
 0.221912   0.713876   0.39562        0.669532   0.107243   0.733368
 0.338986   0.332025   0.307036       0.321163   0.159539   0.0968713
 0.0627475  0.134555   0.433267       0.101195   0.0875229  0.503258
 0.745891   0.0423837  0.943026       0.265498   0.588317   0.458767
 0.527195   0.962525   0.771725    …  0.973972   0.506129   0.931221
 0.758131   0.0534055  0.361368       0.612343   0.107096   0.65761
 0.518219   0.263937   0.0915831      0.42897    0.133383   0.855798
 ⋮                                 ⋱                        ⋮
 

In [15]:
partial_transpose_per(r, 3, (6,6,6))

216×216 Matrix{Float64}:
 0.673292   0.256919   0.534967   …  0.55434    0.752458   0.547227
 0.926011   0.384298   0.395193      0.564659   0.39705    0.219772
 0.35934    0.955374   0.0616085     0.955467   0.721323   0.738936
 0.0682923  0.456162   0.476068      0.357086   0.253345   0.918399
 0.598458   0.217253   0.833675      0.653343   0.381222   0.525958
 0.996734   0.187617   0.100515   …  0.307785   0.305602   0.190131
 0.221912   0.338986   0.0627475     0.406429   0.832959   0.338555
 0.214382   0.0494697  0.937114      0.435746   0.917302   0.378857
 0.398649   0.32614    0.601874      0.540767   0.908673   0.0597125
 0.612473   0.88035    0.914724      0.853798   0.363925   0.358947
 0.776553   0.880377   0.942154   …  0.284668   0.49496    0.148736
 0.05121    0.20365    0.876866      0.591688   0.15698    0.191342
 0.518219   0.462485   0.728345      0.629749   0.706716   0.714103
 ⋮                                ⋱                        ⋮
 0.589566   0.530794   0.3323

In [16]:
log(tr(r))

LoadError: MethodError: no method matching tr(::Array{Float64, 3})

[0mClosest candidates are:
[0m  tr([91m::ITensor[39m; kwargs...)
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/iifFp/src/tensor_operations/[39m[90m[4mmatrix_algebra.jl:19[24m[39m
[0m  tr([91m::MPO[39m; plev, tags)
[0m[90m   @[39m [35mITensors[39m [90m~/.julia/packages/ITensors/iifFp/src/lib/ITensorMPS/src/[39m[90m[4mmpo.jl:542[24m[39m
[0m  tr([91m::Matrix{T}[39m) where T
[0m[90m   @[39m [36mLinearAlgebra[39m [90m~/.julia/juliaup/julia-1.10.3+0.x64.apple.darwin14/share/julia/stdlib/v1.10/LinearAlgebra/src/[39m[90m[4mdense.jl:344[24m[39m
[0m  ...


In [17]:
using ITensors
using LinearAlgebra
let
sites=siteinds("Boson",7;dim=5)
psi=randomMPS(sites; linkdims=10)
psi1=psi

#@show psi1
#reduced density matrix of the 2-nd site
#orthogonalize!(psi1,2)
#rho2=prime(psi1[2],sites[2])*dag(psi1[2])
#@show rho2

#eigenvalues and eigenvectors
#D,U=eigen(rho2)
#@show real(diag(D))

#orthogonalize!(psi1,3)
#two-site reduced density matrix from 2-nd site to 5-th site
#psi1dag=prime(dag(psi1),linkinds(psi1))
#@show psi1dag

#rho25 = prime(psi1[3],linkinds(psi1,1))*prime(psi1dag[3],sites[3])
#for i=4:5
 #rho25 *= psi1[i]
 #rho25 *= psi1dag[i]
#end

#orthogonalize!(psi1,7)
#rho25 *= prime(psi1[7],linkinds(psi1,7))*prime(psi1dag[7],sites[7])
#@show rho25

#eigenvalues and eigenvectors
#D1,U1=eigen(rho25)
#@show real(diag(D1))
#a1=0   
#for j=1:length(diag(real(D1)))
 #  a1+=diag(real(D1))[j]
#end
#@show a1;

 return
end

In [18]:
using ITensors
using LinearAlgebra

function rdm(psi::MPS, a::Int, b::Int)
N = length(psi)
psi1=psi
@assert 1 ≤ a < b ≤ length(psi)
#reduced density matrix of the site a
orthogonalize!(psi1,a)
rhoa=prime(psi1[a],sites[a])*dag(psi1[a])

#reduced density matrix of the site b
orthogonalize!(psi1,b)
rhob=prime(psi1[b],sites[b])*dag(psi1[b])



#two-site reduced density matrix from a site to b site
psi1dag=prime(dag(psi1),linkinds(psi1))

rhoab = prime(psi1[a],linkinds(psi1,a-1))*prime(psi1dag[a],sites[a])
for i=a+1:N
    rhoab *= psi1[i]
    rhoab *= psi1dag[i]
end

rhoab *= prime(psi1[b],linkinds(psi1,b+1))*prime(psi1dag[b],sites[b])
    
d1,u1 = eigen(rhoab)
    
@show diag(real(d1));    

a1=0
for j=1:length(diag(real(d1)))
   a1+=diag(real(d1))[j]
end
@show a1;
return rhoa, rhob, rhoab
end

function entropy(ρ)
D, _ = eigen(ρ)
S = 0.0
for n=1:dim(D, 1)
p = D[n,n]^2
S -= p * log2(p)
end
return S
end

N = 30
sites = siteinds("S=1/2", N)
psi = randomMPS(sites)
rho = outer(psi', psi)
r1, r2, r12 = rdm(psi, 1, 20)

println("S_1", entropy(r1), "S_2",entropy(r2), "S_12",entropy(r12))


diag(real(d1)) = [-2.2825242973068754e-16, 1.2033520269091684e-16, 1.793437193625253e-16, 0.9999999999999997]
a1 = 0.9999999999999998
S_1-6.406853007629836e-16 - 0.0imS_2NaN + NaN*imS_129.610279511444854e-16 - 0.0im


In [19]:
function RDM3(ψ,A,B,C)
    
    
    psi1 = deepcopy(ψ)
    
    orthogonalize!(psi1,A)
    
    # 3-site RDM construction for subsystem = {A} U {B} U {C}.
    # A, B, C denote the sites.

    psi1dag=prime(dag(psi1))

    rho_ABC = prime(psi1[A],s[A])*prime(psi1dag[A],s[A])
    
    for i=A+1:B-1                  
     rho_ABC *= psi1[i]             
     rho_ABC *= psi1dag[i]
    end

    rho_ABC *= prime(psi1[B],s[B])*prime(psi1dag[B],s[B])
    
    for i=B+1:C-1
        rho_ABC *= psi1[i]
        rho_ABC *= psi1dag[i]
    end
    
    rho_ABC *= prime(psi1[C],s[C])*prime(psi1dag[C],s[C])
     
    #@assert hastags(rho_ABC,"Link") == false
    
    DABC,UABC=eigen(rho_ABC,ishermitian=true)
    
    @show length(diag(DABC))
    cc = 0
    for i=1:length(diag(DABC))
        cc += real(diag(DABC))[i]
    end
    @show cc       
    
    return DABC
end

RDM3 (generic function with 1 method)

In [20]:
s = siteinds("S=1/2",10)
p = randomMPS(s,linkdims=4);
RDM3(p,2,3,4)

length(diag(DABC)) = 8
cc = 0.9999999999999994


ITensor ord=2 (dim=8|id=415|"Link,eigen")' (dim=8|id=415|"Link,eigen")
NDTensors.Diag{Float64, Vector{Float64}}